In [1]:
import pandas as pd

#原始数据
df = pd.read_csv("clean_kanzhun_hot_corp_info0.csv")
del df['Unnamed: 0']
len(df) #90
df.head(10) #显示部分数据

,name,salary,score,type,location,scale,url
0,百度,平均工资 ￥10000,4.3,互联网,北京,10000人以上,www.baidu.com/
1,华为,平均工资 ￥11850,4.0,通信设备,深圳,10000人以上,www.huawei.com/cn/
2,富士康,平均工资 ￥4910,3.4,原材料及加工/模具,深圳,1000-9999人,www.foxconn.com.cn
3,中国移动,平均工资 ￥4092,3.7,运营商/增值服务,北京,10000人以上,www.10086.cn
4,腾讯,平均工资 ￥11580,4.4,互联网,深圳,10000人以上,hr.tencent.com
5,阿里巴巴,平均工资 ￥10000,4.3,电子商务,杭州,10000人以上,www.alibabagroup.com
6,京东,平均工资 ￥7720,3.9,电子商务,北京,10000人以上,www.uibe.edu.cn/
7,中国电信,平均工资 ￥3960,3.6,运营商/增值服务,北京,10000人以上,www.chinatelecom.com.cn
8,新东方,平均工资 ￥3915,3.8,学术/科研,北京,1000-9999人,www.xdf.cn
9,中软国际,平均工资 ￥6804,3.0,IT服务/系统集成,北京,1000-9999人,NaN


In [2]:
# 地域分布
df['location'].value_counts()[:7].sum() #83
df['location'].value_counts()

北京     47
上海     13
深圳     12
杭州      4
广州      3
南京      2
青岛      2
香港      1
顺德      1
佛山      1
天津      1
沈阳      1
珠海      1
福州      1
Name: location, dtype: int64

In [3]:
import plotly.plotly as py
import plotly.graph_objs as go

pd_city = pd.read_csv("region_hot_corp_info.csv")
data = [go.Bar(
            x= pd_city['location'],
            y= pd_city['count']
    )]

py.iplot(data, filename='basic-bar')

In [14]:
import plotly.plotly as py
import plotly.graph_objs as go

import pandas as pd

mapbox_access_token = 'pk.eyJ1IjoiaWlpamVubnkiLCJhIjoiY2pwOGJkaDZjMTdqMzNwcDRndWV4Y2IzaCJ9.xprdAkm3-TNaO1MzkvjLyw'

df_location = pd.read_csv("region_hot_corp_info.csv")
site_lat = df_location.lat
site_lon = df_location.lon
locations_name = df_location.location

data = [
    go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=dict(
            size=17,
            color='rgb(255, 0, 0)',
            opacity=0.7
        ),
        text=locations_name,
        hoverinfo='text'
    ),
    go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=dict(
            size=8,
            color='rgb(242, 177, 172)',
            opacity=0.7
        ),
        hoverinfo='none'
    )]

layout = go.Layout(
    title='热门公司分布图',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=38,
            lon=-94
        ),
        pitch=0,
        zoom=3,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='热门公司分布图')

### 这些公司是否有传媒类的呢？

In [6]:
comm = ['媒体','影视','公关','会展','广告','文化','出版','艺术']
count = 0
for i in types0:
    if i in comm:
        count += 1
count

0

In [5]:
# 拆分type信息，并放在同一列上
types = df['type'].str.split('/', expand=True).stack().reset_index(level=1, drop=True)#.rename('type')
types.index = range(1,len(types) + 1) #重新index
len(types) #130
types.value_counts() #没有传媒的影子哟
#df['type'].value_counts()

types0 = types[:]
a = ['计算机软件 ','IT服务','互联网 ','网络设备 ','门户网站']
b = 0
for i in range(1,131):
    if types0[i] in a:
        types0[i] = '互联网公司'
        b = b + 1
b # 30
types0.value_counts()[:10]#.sum()

互联网公司    26
银行        9
通信设备      5
系统集成      5
O2O       4
计算机硬件     4
餐饮        4
电子商务      4
保险        3
家电        3
dtype: int64

In [78]:
import plotly.plotly as py
import plotly.graph_objs as go

fig = {
  "data": [
    {
      "values": [26, 9, 5, 5, 4, 4, 4, 4, 130-61],
      "labels": [
        "互联网公司",
        "银行",
        "通信设备",
        "系统集成",
        "餐饮",
        "电子商务",
        "计算机硬件",
        "O2O",
        "其它"
      ],
      "domain": {"x": [0, .48]},
      "name": "热门公司类型占比",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    },

  ],
  "layout": {
        "title":"公司类型占比",
        "annotations": [
            {
                "font": {
                    "size": 15
                },
                "showarrow": False,
                "text": "热门公司",
                "x": 0.20,
                "y": 0.5
            },

        ]
    }
}
py.iplot(fig, filename='hot_corp_proportion')

In [8]:
len(df['salary']) #90
df['new_salary'] = df['salary'].str.lstrip("平均工资  ￥")

for i in range(0,90):
    df['new_salary'][i] = int(df['new_salary'][i])
df2 = df.sort_values(by='new_salary', ascending=False)[:20]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [10]:
df2

,name,salary,score,type,location,scale,url,new_salary
65,微软中国（Microsoft）,平均工资 ￥14520,4.3,计算机软件,北京,10000人以上,www.microsoft.com,14520
68,爱立信,平均工资 ￥13382,4.1,通信设备,北京,1000-9999人,www.ericsson.com/cn/,13382
22,IBM,平均工资 ￥12035,4.1,IT服务/系统集成,北京,1000-9999人,www.ibm.com/cn/,12035
1,华为,平均工资 ￥11850,4.0,通信设备,深圳,10000人以上,www.huawei.com/cn/,11850
4,腾讯,平均工资 ￥11580,4.4,互联网,深圳,10000人以上,hr.tencent.com,11580
72,GE,平均工资 ￥10485,4.1,IT服务/系统集成,上海,1000-9999人,www.ge.com/cn/,10485
15,兴意达,平均工资 ￥10206,3.6,通信设备,深圳,100-499人,www.shinytel.com,10206
51,惠普（HP）,平均工资 ￥10150,4.1,计算机硬件/网络设备,北京,1000-9999人,NaN,10150
0,百度,平均工资 ￥10000,4.3,互联网,北京,10000人以上,www.baidu.com/,10000
5,阿里巴巴,平均工资 ￥10000,4.3,电子商务,杭州,10000人以上,www.alibabagroup.com,10000


In [9]:
import pandas as pd
import numpy as np
df['new_salary']=pd.DataFrame(df['new_salary'],dtype=np.int)
df['new_salary'].describe()

count       90.000000
mean      6314.811111
std       2560.189740
min       2410.000000
25%       4368.500000
50%       5738.500000
75%       8000.000000
max      14520.000000
Name: new_salary, dtype: float64

In [10]:
import pandas as pd
import numpy as np
df2['new_salary']=pd.DataFrame(df2['new_salary'],dtype=np.int)
df2['new_salary'].describe()

count       20.000000
mean     10045.150000
std       1828.412319
min       8052.000000
25%       8375.000000
50%       9725.000000
75%      10758.750000
max      14520.000000
Name: new_salary, dtype: float64

In [11]:
import plotly.plotly as py
import plotly.graph_objs as go

random_x = df2['name']
random_y0 = df2['new_salary']
random_y1 = df2['score']


# Create a trace
trace0 = go.Scatter(
    x = random_x,
    y = random_y0,
    mode = 'lines+markers',
    name = 'SALARY'
)
trace1 = go.Scatter(
    x = random_x,
    y = random_y1,
    mode = 'lines+markers',
    name = 'GRADE'
)

data = [trace0, trace1]
py.iplot(data, filename='line-mode')

#### 这些热门公司，尤其是互联网公司，有没有我们传媒可以做的岗位呢

In [14]:
df = pd.read_csv("kanzhun_ali.csv")
del df['Unnamed: 0']
df.head(10) #显示部分数据

,name,salary
0,Test Develop Engineer (海外工作学习经验),职位薪资:25000-35000
1,销售经理,职位薪资:7000-14000
2,销售经理,职位薪资:8000-12000
3,UC事业部-高级视觉设计师-RED设计中心,职位薪资:20000-30000
4,乳饮及粮油米面资深招商专员（湖北，江西）,职位薪资:16000-17000
5,销售经理,职位薪资:20000-30000
6,口碑-客户经理（本地生活）-杭州,职位薪资:7000-14000
7,销售顾问,职位薪资:7000-14000
8,推荐产品经理,职位薪资:20000-40000
9,客户经理,职位薪资:9000-13000


In [15]:
df = pd.read_csv("kanzhun_baidu.csv")
del df['Unnamed: 0']
df.head(10) #显示部分数据

,name,salary
0,AI开放平台——JAVA高级研发工程师,职位薪资:20000-40000
1,产品经理,职位薪资:12000-24000
2,IOS平台开发实习生,职位薪资:4000-9000
3,自动驾驶高级算法工程师（感知方向）,职位薪资:1000-2000
4,软件研发工程师（PHP）,职位薪资:25000-50000
5,java开发实习生,职位薪资:4000-5000
6,Python,职位薪资:15000-30000
7,自动驾驶控制算法工程师,职位薪资:1000-2000
8,百度AI实训平台产品经理,职位薪资:20000-40000
9,软件研发工程师（C++）,职位薪资:25000-50000


In [16]:
df = pd.read_csv("kanzhun_tengxun.csv")
del df['Unnamed: 0']
df.head(10) #显示部分数据

,name,salary
0,C++后台开发工程师,职位薪资:20000-40000
1,产品策划,职位薪资:13000-26000
2,腾讯视频数据挖掘工程师,职位薪资:22000-44000
3,数据库引擎高级开发工程师,职位薪资:30000-60000
4,腾讯云云市场前端高级工程师,职位薪资:15000-30000
5,pcg-qq后台开发工程师,职位薪资:20000-40000
6,在线教育Web前端开发高级工程师（深圳）,职位薪资:20000-40000
7,腾讯音乐PHP应用开发工程师,职位薪资:15000-30000
8,MMORPG战斗系统策划师（上海）,职位薪资:15000-30000
9,腾讯医典高级后台开发（深圳）,职位薪资:20000-40000
